In [1]:
#import the necessary libraries
import numpy as np
import pandas as pd
import time
import calendar
import random
import io
import json
import os

In [2]:
dict = {}

In [3]:
#get the current time in unix
gmt = time.gmtime()
current_time  = calendar.timegm(gmt)

In [4]:
current_time

1623134842

In [5]:
#add 2 hours in current time
current_plus_2_hours = calendar.timegm(gmt) + 2 * 60 * 60
#current_plus_2_hours = calendar.timegm(gmt) + 2 * 10

In [6]:
current_plus_2_hours

1623142042

In [7]:
#function to generate random numbers between the given range
def random_rate(rangemin, rangemax):
    return random.randint(rangemin, rangemax)
    

In [8]:
# function to create a dictionary in specified format and a dataframe
def create_dict(user_id,current_time,current_plus_2_hours,i):
    dict['user_id'] = user_id
    dict['timestamp'] = current_time
    dict['heart_rate'] = random_rate(50,170)
    dict['respitation_rate'] = random_rate(30,60)
    dict['activity'] = random_rate(0,10)
    df = pd.DataFrame(dict, index = [i])
    return dict,df
    
    
    

In [9]:
#empty dataframes
df1 = pd.DataFrame()
df2 = pd.DataFrame()
i=0
filename = 'data.json'

if os.path.exists(filename):
    append_write = 'a' # append if already exists
else:
    append_write = 'w' # make a new file if not

file = open(filename,append_write)

#loop will run till current_time is less than current_plus_2_hours
while(current_time < current_plus_2_hours):
    a_dict,df = create_dict('abc',current_time,current_plus_2_hours,i)
    i += 1
    a_dict = json.dumps(a_dict)
    df1 = df1.append(df)
    
    
    file.write(a_dict + '\n')
    current_time += 1

file.close()

In [10]:
df1

,user_id,timestamp,heart_rate,respitation_rate,activity
0,abc,1623134842,117,53,6
1,abc,1623134843,84,42,1
2,abc,1623134844,81,45,6
3,abc,1623134845,125,46,2
4,abc,1623134846,167,51,9
...,...,...,...,...,...
7195,abc,1623142037,165,50,6
7196,abc,1623142038,51,54,4
7197,abc,1623142039,168,52,2
7198,abc,1623142040,139,39,3


In [11]:
df2 = pd.DataFrame()
df3 = pd.DataFrame()

In [12]:
#function to create dataframe output
def create_dataframe(df1,df2, no_segments):
    df2['seg_start'] = df1['timestamp'].groupby(np.arange(len(df1))//no_segments).min()
    df2['seg_end'] = df1['timestamp'].groupby(np.arange(len(df1))//no_segments).max()
    df2['avg_hr'] = df1['heart_rate'].groupby(np.arange(len(df1))//no_segments).mean()
    df2['min_hr'] = df1['heart_rate'].groupby(np.arange(len(df1))//no_segments).min()
    df2['max_hr'] = df1['heart_rate'].groupby(np.arange(len(df1))//no_segments).max()
    df2['avg_rr'] = df1['respitation_rate'].groupby(np.arange(len(df1))//no_segments).mean()
    return df2
    

In [13]:
#calculations for every 15 minutes
no_segments = 15*60
df2 = create_dataframe(df1,df2,no_segments)


In [14]:
df2

,seg_start,seg_end,avg_hr,min_hr,max_hr,avg_rr
0,1623134842,1623135741,110.347778,50,170,45.195556
1,1623135742,1623136641,109.476667,50,170,45.235556
2,1623136642,1623137541,111.313333,50,170,44.851111
3,1623137542,1623138441,109.891111,50,170,45.206667
4,1623138442,1623139341,109.146667,50,170,45.291111
5,1623139342,1623140241,109.791111,50,170,44.930000
6,1623140242,1623141141,110.195556,50,170,45.268889
7,1623141142,1623142041,112.890000,50,170,44.800000


In [15]:
#optional : function to derive hourly avg, min, max, using the above data frame which has 15 mins segments
def create_hourly_dataframe(df2,df3, no_segments):
    df3['seg_start'] = df2['seg_start'].groupby(np.arange(len(df2))//no_segments).min()
    df3['seg_end'] = df2['seg_end'].groupby(np.arange(len(df2))//no_segments).max()
    df3['avg_hr'] = df2['avg_hr'].groupby(np.arange(len(df2))//no_segments).mean()
    df3['min_hr'] = df2['min_hr'].groupby(np.arange(len(df2))//no_segments).min()
    df3['max_hr'] = df2['max_hr'].groupby(np.arange(len(df2))//no_segments).max()
    df3['avg_rr'] = df2['avg_rr'].groupby(np.arange(len(df2))//no_segments).mean()
   
    return df3

In [16]:
no_segments1 = df2.shape[0]//2
df3 = create_hourly_dataframe(df2,df3,no_segments1)

In [17]:
df3

,seg_start,seg_end,avg_hr,min_hr,max_hr,avg_rr
0,1623134842,1623138441,110.257222,50,170,45.122222
1,1623138442,1623142041,110.505833,50,170,45.072500


In [18]:
#save dataframe to output.csv and  output_hourly.csv file
df2.to_csv('output.csv')
df3.to_csv('output_hourly.csv')